# Particle Swarm

Particle swarm optimiztion (PSO) uses an analogy to the foraging behaviour of birds to identify promising areas of the objective. We scatter multiple "particles" which then accelerate towards the best position any of the particles has seen, as well as towards their own historical best position. Eventually, we converge to an answer.

PSO gets stuck in local optima less often than other algorithms such as gradient descent, but it can still miss the global optimum, which becomes more likely if too few particles are used. PSO also allows the "particles" to wander outside of the zone of hypotheses that are actually feasible solutions, so it may sometimes be necessary to modify an objective function so that only valid solutions to the problem score highly (in this notebook, velocity may not be negative).

PSO performs especially well on hard problems with either many local optima or many hypothesis dimensions, but for simple problems it is typically overkill in terms of compuation power spent.

In [ ]:
import random as r
import numpy as np

## Objective Functions

The drag per second a vehicle travelling on a road experiences is approximately R(v) = Av^2 + Bv + C where A is a factor relating aerodynamics, B to rolling and drivetrain losses, and C is a constant load. If t is the time taken to arrive at a destination D, minimizing t x R will minimize the losses due to drag for the journey. Since v = D/t and D is constant this is equivalent to minimizing R/v, i.e. Av + B + C/v. 

Suppose we have a car with A = 0.6, B = 5, C = 80, we have a well-defined optimization problem. It's fun to play around with A/B/C and see how different physical parameters affect the optimum speed. most cars travel ~30mph most of the time, so we expect an answer near 13m/s

In [ ]:
def dragMinimization(
    hypothesis,
    aerodynamicsConstant = 0.6,
    rollingConstant = 5,
    loadConstant = 80
):
    v = hypothesis[0]
    if v < 0:
        return(-float("inf"))
    aTerm = aerodynamicsConstant * v 
    bTerm = rollingConstant
    cTerm = loadConstant / v
    output = - (aTerm + bTerm + cTerm)
    return(output)

## Particle Swarm Optimizer

In [ ]:
class Particle:
        
    def __init__(self,
                objectiveFunction,
                ranges):
        self.ranges = ranges
        self.objectiveFunction = objectiveFunction
        self.currentX = self.generateRandomHypothesis()
        self.ownBestX = self.currentX
        self.currentY = self.objectiveFunction(self.currentX)
        self.ownBestY = self.currentY
        self.velocity = [(0 * x) for x in self.currentX]

    def generateRandomHypothesis(self):
        output = []
        for x in self.ranges:
            output.append(r.uniform(x[0], x[1]))
        return(output)

class ParticleSwarm:
    def __init__(
        self,
        objectiveFunction,
        ranges,
        populationSize = 25
    ):
        self.objectiveFunction = objectiveFunction
        self.ranges = ranges
        self.populationSize = populationSize
        self.population = []
        self.bestY = -float("inf")
        self.bestX = None
        for i in range(populationSize):
            newParticle = Particle(self.objectiveFunction, self.ranges)
            self.population.append(newParticle)
            if newParticle.ownBestY > self.bestY:
                self.bestY = newParticle.ownBestY
                self.bestX = newParticle.ownBestX
    
    def optimize(self,
                objectiveFunction,
                numIterations = 250,
                learnRate = 0.01):
        for i in range(numIterations):
            for j in range(self.populationSize):
                particle = self.population[j]
                updateVector1 = []
                updateVector2 = []
                for k in range(len(self.bestX)):
                    updateVector1.append(-learnRate * (self.bestX[k] - particle.currentX[k]))
                    updateVector2.append(-learnRate * (particle.ownBestX[k] - particle.currentX[k]))
                    particle.velocity[k] += updateVector1[k]
                    particle.velocity[k] += updateVector2[k]
                    particle.currentX[k] += particle.velocity[k]
                    particle.currentY = self.objectiveFunction(particle.currentX)
                    if particle.currentY > particle.ownBestY:
                        particle.ownBestY = particle.currentY
                        particle.ownBestX = particle.currentX
                    if particle.currentY > self.bestY:
                        self.bestY = particle.currentY
                        self.bestX = particle.currentX
        return(self.bestX)

ps = ParticleSwarm(dragMinimization, [[0, 25]])
print(ps.optimize(dragMinimization))